In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(conf=SparkConf().setAppName("MyApp").setMaster("local"))

In [2]:
# Read the file
def parse_edge(s):
    user, follower = s.split("\t")
    return (int(user), int(follower))

n = 400  # number of partitions
edges = sc.textFile("/data/twitter/twitter_sample_small.txt").map(parse_edge).cache()
forward_edges = edges.map(lambda e: (e[1], e[0])).partitionBy(n).persist()

start_node = 12
end_node = 34

In [3]:
def attach_node(x):
    return (x[1][1], x[1][0] + [x[1][1]])

In [21]:
paths = sc.parallelize([(start_node, [start_node, ])])
counter = 0
while True:
    
    paths = paths.join(forward_edges).map(attach_node)
    completed_paths = paths.filter(lambda x: x[0] == end_node)
    if completed_paths.count() > 0:
        print(completed_paths.collect())
        break
    
    counter += 1
    print(counter)
    print(paths.collect())
    #if counter == 2:
    #    break

1
[(126, [12, 126]), (380, [12, 380]), (422, [12, 422]), (648, [12, 648])]
2
[(53, [12, 422, 53])]
3
[(31, [12, 422, 53, 31]), (52, [12, 422, 53, 52]), (57, [12, 422, 53, 57]), (150, [12, 422, 53, 150]), (187, [12, 422, 53, 187]), (292, [12, 422, 53, 292]), (652, [12, 422, 53, 652])]
4
[(13, [12, 422, 53, 31, 13]), (107, [12, 422, 53, 52, 107])]
5
[(15, [12, 422, 53, 52, 107, 15]), (20, [12, 422, 53, 52, 107, 20]), (586, [12, 422, 53, 52, 107, 586])]
6
[(23, [12, 422, 53, 52, 107, 20, 23]), (87, [12, 422, 53, 52, 107, 20, 87]), (292, [12, 422, 53, 52, 107, 20, 292]), (295, [12, 422, 53, 52, 107, 20, 295]), (107, [12, 422, 53, 52, 107, 586, 107])]
7
[(16, [12, 422, 53, 52, 107, 20, 23, 16]), (38, [12, 422, 53, 52, 107, 20, 23, 38]), (274, [12, 422, 53, 52, 107, 20, 23, 274]), (64, [12, 422, 53, 52, 107, 20, 87, 64]), (15, [12, 422, 53, 52, 107, 586, 107, 15]), (20, [12, 422, 53, 52, 107, 586, 107, 20]), (586, [12, 422, 53, 52, 107, 586, 107, 586])]
[(34, [12, 422, 53, 52, 107, 20, 23, 2

In [36]:
res = completed_paths.take(1)
print(','.join([str(x) for x in res[0][1]]))

12,422,53,52,107,20,23,274,34
